# VacationPy
----

## Please read:

* Cells that contain heatmaps have commented out pieces of code. Uncomment those for map visualization.
* Dependencies below may not be installed in your environment. Please install them using !pip install <package_name> code in the cell below.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from IPython.display import HTML
from ipywidgets.embed import embed_minimal_html
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("../WeatherPy/cities.csv", index_col=False)
cities_df

,Date,Cloudiness,Wind Speed,Lat,Lng,Max Temp,Country,City,Humidity,parsed_time
0,1580071598,40,5.82,-21.20,-175.20,80.60,TO,Vaini,88,2020-01-26 20:46:38
1,1580071696,100,3.56,-0.60,73.08,80.83,MV,Hithadhoo,77,2020-01-26 20:48:16
2,1580071732,0,1.19,36.52,46.21,11.07,IR,Bukan,83,2020-01-26 20:48:52
3,1580071642,1,9.17,71.29,-156.79,-23.80,US,Barrow,74,2020-01-26 20:47:22
4,1580071736,20,6.93,-34.15,19.02,63.00,ZA,Grabouw,72,2020-01-26 20:48:56
...,...,...,...,...,...,...,...,...,...,...
544,1580072398,100,1.70,50.59,107.60,0.93,RU,Bichura,83,2020-01-26 20:59:58
545,1580072399,100,2.17,-4.24,142.84,74.64,PG,Ambunti,99,2020-01-26 20:59:59
546,1580072400,20,5.82,51.03,-4.22,46.99,GB,Northam,100,2020-01-26 21:00:00
547,1580072401,0,12.75,-23.67,-46.46,84.20,BR,Maua,28,2020-01-26 21:00:01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

#custom layour for the figure display

figure_layout = {
'width': '800px',
'height': '600px',
'border': '1px solid black',
'padding': '1px'
}

# the center of the map is selected to be in Rome based for better visualization of the map. The zoom level is set to 1.6 to 
# cut out the excess portions of the World map.

fig = gmaps.figure(layout=figure_layout, zoom_level=1.6, center=[41.9028,12.4964])
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['Lat', 'Lng']], weights=cities_df['Humidity'],
    max_intensity=30, point_radius=20.0)
fig.add_layer(heatmap_layer)
fig
#embed_minimal_html('map.html', views=[fig])
#HTML(filename="./map.html")

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Favorable weather conditions for summer vacation: (Could use other weather conditions fbased on personal preference)

* A max temperature lower than 80 degrees but higher than 70.

* Wind speed less than 10 mph.

* Zero cloudiness. This requirement is too strict for this period of time, therefore in conditions below the cloudiness was
set to be less than 10%


In [5]:
#null and nan variable were not included in this database from the beginning.

cloudiness_mask = cities_df["Cloudiness"] <= 10
windSpeed_mask = cities_df['Wind Speed'] < 10
maxTemp_mask = (cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters. I set the radius of search to 6000 since sometimes 5000 did not contain any hotels.   
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
narrowed_city_df = cities_df[cloudiness_mask & windSpeed_mask & maxTemp_mask]
hotel_df = pd.DataFrame()

for index, row in narrowed_city_df.iterrows():
    try:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={narrowed_city_df.loc[index, 'Lat']},{narrowed_city_df.loc[index,'Lng']}&radius=6000&type=lodging&keyword=hotel&key={g_key}"
        response = requests.get(url).json()
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = response['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']
        print(f"Successfully fetched information about hotel near {row['City']}")
    except:
        print(f"Information about hotel near {row['City']} could not be fetched")

Successfully fetched information about hotel near Yulara
Successfully fetched information about hotel near Mossamedes
Successfully fetched information about hotel near Castro
Successfully fetched information about hotel near Celestun
Information about hotel near Beloha could not be fetched
Information about hotel near Taoudenni could not be fetched
Successfully fetched information about hotel near Taltal
Successfully fetched information about hotel near Ewo
Successfully fetched information about hotel near Nsanje
Information about hotel near Barra Patuca could not be fetched
Information about hotel near Chiltepec could not be fetched
Successfully fetched information about hotel near Kousseri
Successfully fetched information about hotel near Bonthe
Successfully fetched information about hotel near Ahipara
Successfully fetched information about hotel near Kaeo
Successfully fetched information about hotel near Bahia Honda
Successfully fetched information about hotel near Bosaso
Successful

In [7]:
# The upper code will leave the original indexing of the cities. Therfore below code is meant to reset the index to start from 0
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Desert Gardens Hotel - Ayers Rock Resort,-25.243725,130.984807,Yulara,AU
1,Hotel ibis Styles Iu Namibe,-15.190917,12.151250,Mossamedes,AO
2,Hotel Buganville Express,-24.789677,-50.006275,Castro,BR
3,Playa Maya Resorts,20.923683,-90.377354,Celestun,MX
4,Hotel Mi Tampi,-25.405057,-70.482855,Taltal,CL
5,Hotel D'Ewo,-0.866667,14.816667,Ewo,CG
6,Nsanje Discovery Lodge,-16.923563,35.257581,Nsanje,MW
7,Hilton N'Djamena,12.089108,15.065662,Kousseri,CM
8,Bonthe Holiday Village,7.525635,-12.500587,Bonthe,SL
9,Endless Summer Lodge - BBH,-35.178090,173.134497,Ahipara,NZ


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template ="""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Desert Gardens Hotel - Ayers Rock Resort</dd>\n<dt>City</dt><dd>Yulara</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel ibis Styles Iu Namibe</dd>\n<dt>City</dt><dd>Mossamedes</dd>\n<dt>Country</dt><dd>AO</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Buganville Express</dd>\n<dt>City</dt><dd>Castro</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Playa Maya Resorts</dd>\n<dt>City</dt><dd>Celestun</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Mi Tampi</dd>\n<dt>City</dt><dd>Taltal</dd>\n<dt>Country</dt><dd>CL</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Hotel D'Ewo</dd>\n<dt>City</dt><dd>Ewo</dd>\n<dt>Country</dt><dd>CG</dd>\n</dl>\n",
 '\n<dl>\n<dt>Name</dt><dd>Nsanje Discovery Lodge</dd>\n<dt>City</dt><dd>Nsanje</dd>\n<dt>Country</dt><dd>MW</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Hilton N'Djamena</dd>\n<dt>City</dt><dd>Kousseri</dd>\n<dt>Country</dt><dd>CM</dd>\n</dl>\n",

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map
#embed_minimal_html('vacation_map.html', views=[fig])
#HTML(filename="./vacation_map.html")

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))